# Session 14: Assembly techniques
Made by:

**- Ruelas Flores, César Diego**

- **Environment Setup:**

In [1]:
# --- Configuración General ---
import os
import sys
import logging
import pyarrow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- Configuración del Logging ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)

## 1. VARIABLES GLOBALES

In [2]:
# TARGET_COLUMN: La variable que queremos predecir.
TARGET_COLUMN = 'actual_productivity'

# TEST_SIZE: Proporción del dataset que se usará para el conjunto de prueba.
TEST_SIZE = 0.2

# RANDOM_STATE: Semilla para la aleatoriedad, asegura reproducibilidad.
RANDOM_STATE = 42

## 2. FUNCIONES (../src/utils.py)

In [3]:
%%writefile ../src/utils.py
import logging
import pyarrow
from typing import List, Tuple, Dict, Any

import pandas as pd
import numpy as np
import polars as pl
from ucimlrepo import fetch_ucirepo

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, BaggingRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def load_and_prepare_data() -> pd.DataFrame:
    """
    Carga el dataset de UCI, lo combina y lo retorna como un DataFrame de pandas.
    Usa Polars internamente para una operación de limpieza rápida.

    Returns:
        pd.DataFrame: El DataFrame de pandas listo para el preprocesamiento.
    """
    logging.info("Iniciando la carga de datos desde el repositorio UCI...")
    try:
        garment_prod = fetch_ucirepo(id=597)
        X_pd = garment_prod.data.features
        y_pd = garment_prod.data.targets
        
        df_pd = pd.concat([X_pd, y_pd], axis=1)
        
        # --- Uso de Polars para una operación eficiente ---
        # Convertimos a Polars para eliminar la columna de forma rápida y segura
        df_pl = pl.from_pandas(df_pd)
        if 'date' in df_pl.columns:
            df_pl = df_pl.drop('date')
            logging.info("Columna 'date' eliminada usando Polars.")
            
        # Regresamos a pandas para continuar el flujo de trabajo
        df_final_pd = df_pl.to_pandas()
        # -------------------------------------------------
        
        logging.info(f"Datos cargados exitosamente. Dimensiones del DataFrame: {df_final_pd.shape}")
        return df_final_pd
    except Exception as e:
        logging.error(f"Error al cargar o preparar los datos: {e}")
        raise

def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Imputa los valores faltantes en un DataFrame de pandas usando la mediana.

    Args:
        df (pd.DataFrame): El DataFrame con posibles valores nulos.

    Returns:
        pd.DataFrame: El DataFrame sin valores nulos.
    """
    logging.info("Iniciando tratamiento de datos faltantes.")
    missing_counts = df.isnull().sum().sum()
    
    if missing_counts == 0:
        logging.info("No se encontraron valores faltantes.")
        return df

    logging.info(f"Total de valores faltantes encontrados: {missing_counts}")
    
    # Iterar sobre columnas con nulos y rellenar con la mediana
    for col in df.columns:
        if df[col].isnull().any():
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)
            logging.info(f"Valores nulos en '{col}' imputados con la mediana ({median_val}).")
    
    return df

def treat_outliers_iqr(df: pd.DataFrame, num_cols: List[str]) -> pd.DataFrame:
    """
    Trata los outliers en columnas numéricas usando el método de capping por IQR.

    Args:
        df (pd.DataFrame): El DataFrame de entrada.
        num_cols (List[str]): Lista de columnas numéricas a tratar.

    Returns:
        pd.DataFrame: DataFrame con outliers tratados.
    """
    logging.info("Iniciando tratamiento de outliers a nivel univariado (IQR).")
    df_out = df.copy()
    
    for col in num_cols:
        Q1 = df_out[col].quantile(0.25)
        Q3 = df_out[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers_count = ((df_out[col] < lower_bound) | (df_out[col] > upper_bound)).sum()
        
        if outliers_count > 0:
            logging.info(f"Tratando {outliers_count} outliers en la columna '{col}'.")
            df_out[col] = np.clip(df_out[col], lower_bound, upper_bound)
            
    logging.info("Tratamiento de outliers finalizado.")
    return df_out

def convert_to_dummies(df: pd.DataFrame, cat_cols: List[str]) -> pd.DataFrame:
    """
    Convierte las columnas categóricas a variables dummy usando pandas.

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        cat_cols (List[str]): Lista de columnas categóricas.

    Returns:
        pd.DataFrame: DataFrame con variables dummy.
    """
    logging.info("Convirtiendo variables categóricas a dummies...")
    if not cat_cols:
        logging.info("No hay columnas categóricas para convertir.")
        return df
    
    df_dummies = pd.get_dummies(df, columns=cat_cols, drop_first=True, dtype=float)
    logging.info(f"Columnas convertidas a dummies: {cat_cols}")
    return df_dummies

def train_and_evaluate_models(
    X_train: pd.DataFrame, 
    y_train: pd.Series, 
    X_test: pd.DataFrame, 
    y_test: pd.Series, 
    random_state: int
) -> pd.DataFrame:
    """
    Entrena y evalúa varios modelos de ensamblaje.

    Args:
        X_train (pd.DataFrame): Características de entrenamiento.
        y_train (pd.Series): Objetivo de entrenamiento.
        X_test (pd.DataFrame): Características de prueba.
        y_test (pd.Series): Objetivo de prueba.
        random_state (int): Semilla aleatoria para reproducibilidad.

    Returns:
        pd.DataFrame: Un DataFrame con las métricas de evaluación para cada modelo.
    """
    logging.info("Iniciando entrenamiento y evaluación de modelos de ensamblaje.")
    
    # --- Definición de modelos base con hiperparámetros ---
    r1 = LinearRegression()
    r2 = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=random_state)
    r3 = KNeighborsRegressor(n_neighbors=10)
    r4 = SVR(kernel='rbf', C=1.0)
    r5 = DecisionTreeRegressor(max_depth=5, random_state=random_state)

    # --- Modelos de Ensamblaje ---
    models = {
        'Voting': VotingRegressor(estimators=[('lr', r1), ('rf', r2), ('knn', r3)]),
        'Bagging': BaggingRegressor(estimator=r5, n_estimators=50, random_state=random_state),
        'Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=random_state),
        'Stacking': StackingRegressor(estimators=[('rf', r2), ('knn', r3), ('svr', r4)], final_estimator=r1)
    }
    
    results = []
    for name, model in models.items():
        logging.info(f"Entrenando el modelo: {name}...")
        model.fit(X_train, y_train)
        
        logging.info(f"Evaluando el modelo: {name}...")
        y_pred = model.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        results.append({
            "Modelo": name,
            "MSE": mse,
            "RMSE": np.sqrt(mse),
            "MAE": mean_absolute_error(y_test, y_pred),
            "R2 Score": r2_score(y_test, y_pred)
        })
    logging.info("Evaluación de todos los modelos completada.")
    return pd.DataFrame(results)

Overwriting ../src/utils.py


## 3. IMPLEMENTACIÓN PRINCIPAL

In [4]:
logging.info("--- INICIANDO LA IMPLEMENTACIÓN PRINCIPAL ---")

2025-06-19 15:20:23,980 - INFO - --- INICIANDO LA IMPLEMENTACIÓN PRINCIPAL ---


--- Configuración de la ruta del proyecto ---

In [5]:
# --- Configuración de la ruta del proyecto ---
# Esto es crucial para poder importar nuestro módulo 'utils'.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
        logging.info(f"Ruta del proyecto '{project_root}' agregada al sys.path.")
    
    # Esta importación ahora funcionará porque la celda anterior creó el archivo.
    from src import utils
    logging.info("Módulo 'utils' importado exitosamente.")
    
except Exception as e:
    logging.error(f"Error en la configuración inicial: {e}")

2025-06-19 15:20:25,644 - INFO - Ruta del proyecto 'c:\Users\AzShet\Documents\Jupyter_LAB\jupyter_projects\5to_ciclo\DataMining\lab14\Data_Mining-LAB14' agregada al sys.path.
2025-06-19 15:20:26,180 - INFO - Módulo 'utils' importado exitosamente.


In [ ]:
# display(df)


### 3.1 Parte A: Preprocesamiento

In [6]:
logging.info("--- [PARTE A] Iniciando preprocesamiento ---")
# Cargar datos usando la función de utils
df = utils.load_and_prepare_data()

2025-06-19 15:20:31,714 - INFO - --- [PARTE A] Iniciando preprocesamiento ---
2025-06-19 15:20:31,716 - INFO - Iniciando la carga de datos desde el repositorio UCI...
2025-06-19 15:20:33,383 - INFO - Columna 'date' eliminada usando Polars.
2025-06-19 15:20:33,391 - INFO - Datos cargados exitosamente. Dimensiones del DataFrame: (1197, 14)


In [7]:
# Identificar columnas numéricas y categóricas (excluyendo el target)
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = df.select_dtypes(include=np.number).columns.tolist()
num_cols.remove(TARGET_COLUMN)

logging.info(f"Columnas categóricas: {cat_cols}")
logging.info(f"Columnas numéricas: {num_cols}")

2025-06-19 15:20:55,428 - INFO - Columnas categóricas: ['quarter', 'department', 'day']
2025-06-19 15:20:55,430 - INFO - Columnas numéricas: ['team', 'targeted_productivity', 'smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers']


In [8]:
# Pipeline de preprocesamiento usando las funciones de utils
df_processed = (
    df.pipe(utils.handle_missing_values)
    .pipe(utils.convert_to_dummies, cat_cols=cat_cols)
)

2025-06-19 15:20:58,894 - INFO - Iniciando tratamiento de datos faltantes.
2025-06-19 15:20:58,897 - INFO - Total de valores faltantes encontrados: 506
2025-06-19 15:20:58,901 - INFO - Valores nulos en 'wip' imputados con la mediana (1039.0).
2025-06-19 15:20:58,904 - INFO - Convirtiendo variables categóricas a dummies...
2025-06-19 15:20:58,915 - INFO - Columnas convertidas a dummies: ['quarter', 'department', 'day']


In [9]:
# Las nuevas columnas dummy son numéricas, actualizamos la lista para tratar outliers
final_num_cols = df_processed.select_dtypes(include=np.number).columns.tolist()
final_num_cols.remove(TARGET_COLUMN)

df_processed = utils.treat_outliers_iqr(df_processed, num_cols=final_num_cols)
logging.info("--- [PARTE A] Preprocesamiento finalizado ---")

2025-06-19 15:21:09,519 - INFO - Iniciando tratamiento de outliers a nivel univariado (IQR).
2025-06-19 15:21:09,527 - INFO - Tratando 79 outliers en la columna 'targeted_productivity'.
2025-06-19 15:21:09,534 - INFO - Tratando 358 outliers en la columna 'wip'.
2025-06-19 15:21:09,542 - INFO - Tratando 1 outliers en la columna 'over_time'.
2025-06-19 15:21:09,547 - INFO - Tratando 11 outliers en la columna 'incentive'.
2025-06-19 15:21:09,551 - INFO - Tratando 18 outliers en la columna 'idle_time'.
2025-06-19 15:21:09,557 - INFO - Tratando 18 outliers en la columna 'idle_men'.
2025-06-19 15:21:09,564 - INFO - Tratando 147 outliers en la columna 'no_of_style_change'.
2025-06-19 15:21:09,578 - INFO - Tratando 210 outliers en la columna 'quarter_Quarter3'.
2025-06-19 15:21:09,582 - INFO - Tratando 248 outliers en la columna 'quarter_Quarter4'.
2025-06-19 15:21:09,587 - INFO - Tratando 44 outliers en la columna 'quarter_Quarter5'.
2025-06-19 15:21:09,594 - INFO - Tratando 187 outliers en l

### 3.2. Parte B: Separación y Escalado de Datos

In [10]:
logging.info("--- [PARTE B] Iniciando separación y escalado ---")

2025-06-19 15:21:51,374 - INFO - --- [PARTE B] Iniciando separación y escalado ---


In [11]:
# Separar características (X) y objetivo (y)
X = df_processed.drop(columns=TARGET_COLUMN)
y = df_processed[TARGET_COLUMN]

In [12]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
logging.info(f"Datos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")

2025-06-19 15:21:56,498 - INFO - Datos divididos: 957 para entrenamiento, 240 para prueba.


In [13]:
# Escalar características numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Convertir de vuelta a DataFrame para mantener la consistencia
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)
logging.info("--- [PARTE B] Escalado de características finalizado ---")

2025-06-19 15:22:00,213 - INFO - --- [PARTE B] Escalado de características finalizado ---


### 3.3. Parte C: Entrenamiento y Evaluación de Modelos

In [15]:
logging.info("--- [PARTE C] Iniciando entrenamiento y evaluación de modelos ---")
model_results_df = utils.train_and_evaluate_models(
    X_train, y_train, X_test, y_test, random_state=RANDOM_STATE
)
logging.info("--- [PARTE C] Evaluación de modelos finalizada ---")

2025-06-19 15:22:08,194 - INFO - --- [PARTE C] Iniciando entrenamiento y evaluación de modelos ---
2025-06-19 15:22:08,195 - INFO - Iniciando entrenamiento y evaluación de modelos de ensamblaje.
2025-06-19 15:22:08,196 - INFO - Entrenando el modelo: Voting...
2025-06-19 15:22:08,384 - INFO - Evaluando el modelo: Voting...
2025-06-19 15:22:13,015 - INFO - Entrenando el modelo: Bagging...
2025-06-19 15:22:13,186 - INFO - Evaluando el modelo: Bagging...
2025-06-19 15:22:13,199 - INFO - Entrenando el modelo: Boosting...
2025-06-19 15:22:13,357 - INFO - Evaluando el modelo: Boosting...
2025-06-19 15:22:13,362 - INFO - Entrenando el modelo: Stacking...
2025-06-19 15:22:14,382 - INFO - Evaluando el modelo: Stacking...
2025-06-19 15:22:14,418 - INFO - Evaluación de todos los modelos completada.
2025-06-19 15:22:14,421 - INFO - --- [PARTE C] Evaluación de modelos finalizada ---


### 3.4. **Visualización de Resultados**

In [19]:
#-----------------------------------------------------------------------------
logging.info("--- RESULTADOS FINALES DE LA EVALUACIÓN ---")

# Ordenar resultados por R2 Score para una mejor visualización
model_results_df_sorted = model_results_df.sort_values(by="R2 Score", ascending=False).reset_index(drop=True)

# Mostrar la tabla de resultados en el output del notebook
display(model_results_df_sorted)

2025-06-19 15:24:33,805 - INFO - --- RESULTADOS FINALES DE LA EVALUACIÓN ---


,Modelo,MSE,RMSE,MAE,R2 Score
0,Bagging,0.013348,0.115532,0.077975,0.497309
1,Stacking,0.014207,0.119194,0.079295,0.464935
2,Boosting,0.014493,0.120388,0.078522,0.454163
3,Voting,0.014779,0.121570,0.079825,0.443392


In [25]:

# Justificación del mejor modelo
best_model = model_results_df_sorted.iloc[0]
logging.info(f"""
--- CONCLUSIÓN ---
El mejor modelo es '{best_model['Modelo']}' con los siguientes resultados:
- R2 Score: {best_model['R2 Score']:.4f}
- RMSE:     {best_model['RMSE']:.4f}

Justificación: El modelo '{best_model['Modelo']}' se elige como el mejor al presentar el coeficiente de determinación (R2 Score) más alto.
Esto indica que es el modelo que mejor explica la variabilidad de la productividad real de los empleados entre todas las opciones evaluadas.
""")

2025-06-19 16:21:37,770 - INFO - 
--- CONCLUSIÓN ---
El mejor modelo es 'Bagging' con los siguientes resultados:
- R2 Score: 0.4973
- RMSE:     0.1155

Justificación: El modelo 'Bagging' se elige como el mejor al presentar el coeficiente de determinación (R2 Score) más alto.
Esto indica que es el modelo que mejor explica la variabilidad de la productividad real de los empleados entre todas las opciones evaluadas.



## 4. TESTING (../tests/test_utils.py)

In [16]:
%%writefile ../tests/test_utils.py
import pytest
import pandas as pd
import numpy as np
import sys
import os

# Añadir el directorio raíz del proyecto al path para encontrar el módulo src
project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src import utils

@pytest.fixture
def sample_df() -> pd.DataFrame:
    """Fixture que crea un DataFrame de pandas de ejemplo para las pruebas."""
    data = {
        'num_col1': [1, 2, 3, 4, 100],
        'num_col2': [10.0, 20.0, np.nan, 40.0, 50.0],
        'cat_col': ['A', 'B', 'A', 'C', 'B'],
        'target': [1.1, 2.2, 3.3, 4.4, 5.5]
    }
    return pd.DataFrame(data)

def test_load_and_prepare_data():
    """Prueba que la carga de datos funciona y elimina la columna 'date'."""
    df = utils.load_and_prepare_data()
    assert isinstance(df, pd.DataFrame)
    assert 'date' not in df.columns
    assert 'actual_productivity' in df.columns

def test_handle_missing_values(sample_df):
    """Prueba que los valores nulos son imputados correctamente."""
    df_imputed = utils.handle_missing_values(sample_df)
    assert df_imputed['num_col2'].isnull().sum() == 0
    # La mediana de [10, 20, 40, 50] es 30.0
    assert df_imputed.loc[2, 'num_col2'] == 30.0

def test_convert_to_dummies(sample_df):
    """Prueba la conversión a variables dummy."""
    df_dummies = utils.convert_to_dummies(sample_df, cat_cols=['cat_col'])
    assert 'cat_col' not in df_dummies.columns
    assert 'cat_col_B' in df_dummies.columns
    assert 'cat_col_C' in df_dummies.columns
    assert 'cat_col_A' not in df_dummies.columns

def test_treat_outliers_iqr(sample_df):
    """Prueba que los outliers son tratados (capped)."""
    num_cols = ['num_col1']
    df_treated = utils.treat_outliers_iqr(sample_df.copy(), num_cols)
    Q1 = sample_df['num_col1'].quantile(0.25) # 2.5
    Q3 = sample_df['num_col1'].quantile(0.75) # 4.5 -> This is incorrect for [1,2,3,4,100] pandas quantile is different. Let's calculate manually.
    # Sorted: [1, 2, 3, 4, 100]. Q1=2.0, Q3=4.0, IQR=2.0. Upper bound = 4.0 + 1.5*2.0 = 7.0
    upper_bound = 4.0 + 1.5 * (4.0 - 2.0)
    assert df_treated.loc[4, 'num_col1'] == upper_bound

Overwriting ../tests/test_utils.py


## 5. EJECUCIÓN

In [21]:
!pytest ../ -v -p no:cacheprovider

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.5.0 -- C:\Users\AzShet\AppData\Local\Programs\Python\Python313\python.exe
rootdir: c:\Users\AzShet\Documents\Jupyter_LAB\jupyter_projects\5to_ciclo\DataMining\lab14\Data_Mining-LAB14
plugins: anyio-4.9.0, mock-3.14.1
collecting ... collected 4 items

..\tests\test_utils.py::test_load_and_prepare_data PASSED                [ 25%]
..\tests\test_utils.py::test_handle_missing_values PASSED                [ 50%]
..\tests\test_utils.py::test_convert_to_dummies PASSED                   [ 75%]
..\tests\test_utils.py::test_treat_outliers_iqr PASSED                   [100%]

============================== 4 passed in 6.73s ==============================
